<a href="https://colab.research.google.com/github/hoangvuduyanh33/QA/blob/main/notebooks/DrQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Presequisite

## Install coccoc-tokenizer

In [ ]:
%cd /content
!git clone -q https://github.com/coccoc/coccoc-tokenizer

%cd coccoc-tokenizer
!mkdir build
%cd build
!cmake -DBUILD_PYTHON=1 ..
!make install
!mv /usr/local/lib/python3.7/site-packages/CocCocTokenizer* /usr/local/lib/python3.7/dist-packages

## Install DrQA

In [ ]:
%cd /content
!git clone -q --recurse-submodules https://github.com/hoangvuduyanh33/QA
%cd /content/QA
!pip install -q -r requirements.txt
!python setup.py -q develop

## Download data

In [ ]:
%cd /content/QA
!mkdir data

In [ ]:
%cd /tmp
!gdown https://drive.google.com/uc?id=1be4DVbFF9rp3oua7rBZ3O4wEDSsg045m -O datasets.zip
!gdown https://drive.google.com/uc?id=1--6kxO1fUwScvn3CFkR8_XzA4s63Tevf -O wikipedia_using.zip
!gdown https://drive.google.com/uc?id=1Qjb5MzhRGjdKpoDI3vweDRdfWP6oxdid -O english_wiki_2018_index.zip
!gdown https://drive.google.com/uc?id=12tvYRn8oHK0VXKZiqaubhkij348PxRTM -O wvietnamese_wiki_20210501_index.zip

In [ ]:
%cd /content/QA
!unzip -q /tmp/datasets.zip ./data
!unzip -q /tmp/wikipedia_using.zip ./data
!unzip -q /tmp/english_wiki_2018_index.zip -d ./data
!unzip -q /tmp/vietnamese_wiki_20210501_index.zip -d ./data/index

## Download model

In [ ]:
%cd /content/QA/data
!mkdir model

In [ ]:
%cd /tmp
!gdown https://drive.google.com/uc?id=1-P-uPD43Qb_hP45JTeVkrPqNstLrXZ8Z -O output-xlm-roberta-base-mlqa-xquad-vi-wiki.zip

In [ ]:
%cd /content/QA/data
!unzip -q /tmp/output-xlm-roberta-base-mlqa-xquad-vi-wiki.zip ./model

# Run interactive pipeline

## English

### Pyserini

In [ ]:
%cd /content/QA

!python scripts/pipeline_transformers/interactive.py \
    --retriever serini-bm25 \
    --reader-model distilbert-base-cased-distilled-squad \
    --index-path data/index/lucene-index.enwiki-20180701-paragraphs \
    --num-workers 6 \
    --index-lan en

### DrQA-Transformers

In [ ]:
%cd /content/QA

!python scripts/pipeline_transformers/interactive.py \
    --retriever-model data/wikipedia_using/en/docs-tfidf-ngram=1-hash=16777216-tokenizer=spacy.npz \
    --doc-db data/wikipedia_using/en/docs.db \
    --num-workers 6 \
    --retriever tfidf

## Vietnamese

### Pyserini

In [ ]:
%cd /content/QA

!python scripts/pipeline_transformers/interactive.py \
    --retriever serini-bm25 \
    --reader-model data/model/output-xlm-roberta-base-mlqa-xquad-vi-wiki \
    --index-path data/index/lucene-index.viwiki-20210501-paragraphs \
    --num-workers 6 \
    --index-lan vi

### DrQA-Transformers

In [ ]:
%cd /content/QA

!python scripts/pipeline_transformers/interactive.py \
    --retriever-model data/wikipedia_using/vi/docs-tfidf-ngram=1-hash=16777216-tokenizer=coccoc.npz \
    --doc-db data/wikipedia_using/vi/docs.db \
    --num-workers 6 \
    --retriever tfidf

# Run WebUI

## English

### Pyserini

In [ ]:
%cd /content/QA/drqa-webui
!USE_NGROK=True gunicorn --timeout 300 "index:app(retriever='serini-bm25', lang='en')"

### DrQA-Transformers

In [ ]:
%cd /content/QA/drqa-webui
!USE_NGROK=True gunicorn --timeout 300 "index:app(retriever='tfidf', lang='en')"

## Vietnamese

### Pyserini

In [ ]:
%cd /content/QA/drqa-webui
!USE_NGROK=True gunicorn --timeout 300 "index:app(retriever='serini-bm25', lang='vi')"

### DrQA-Transformers

In [ ]:
%cd /content/QA/drqa-webui
!USE_NGROK=True gunicorn --timeout 300 "index:app(retriever='tfidf', lang='vi')"